In [1]:
import pandas as pd
import numpy as np
from scipy.fft import fft, ifft, fftfreq

In [2]:
# file_paths = {
#     "balance": "hc100_balance.csv",
#     "hurriedpace": "hc100_hurriedpace.csv",
#     "hurriedpace_mat": "hc100_hurriedpace_mat.csv",
#     "selfpace": "hc100_selfpace.csv",
#     "selfpace_mat": "hc100_selfpace_mat.csv",
#     "selfpace_matturn": "hc100_selfpace_matturn.csv",
#     "tandemgait": "hc100_tandemgait.csv",
#     "tug": "hc100_tug.csv"
# }

### CURRENTLY IT USES FILE NAME COMING FROM FILES_JOIN.PY. If want to use directly in this change the file_paths.


print("###############################################")
print(file_name)
print("###############################################")
# Load all files into DataFrames
# dataframes = {name: pd.read_csv(path) for name, path in file_paths.items()}

# # Display basic information about each file
# file_overview = {name: df.info() for name, df in dataframes.items()}

NameError: name 'csv_file_paths' is not defined

In [123]:
# selfpace_data = dataframes["selfpace"]
selfpace_data = pd.read_csv(file_name)

In [124]:
if 'Time' in selfpace_data.columns:
    selfpace_data['Time'] = selfpace_data['Time'].str.replace(' sec', '', regex=False).astype(float)

In [125]:
relevant_columns = ['L Foot Contact', 'R Foot Contact', 'Walkway_X', 'Walkway_Y']
selfpace_data[relevant_columns] = selfpace_data[relevant_columns].fillna(method='ffill')

# 1.3 Expand 'Walkway_X' and 'Walkway_Y' values split by "|"
walkway_x_patterns = selfpace_data['Walkway_X'].str.split('|', expand=True).astype(float)
walkway_y_patterns = selfpace_data['Walkway_Y'].str.split('|', expand=True).astype(float)


C:\Users\dhruv\AppData\Local\Temp\ipykernel_5392\1887272773.py:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  selfpace_data[relevant_columns] = selfpace_data[relevant_columns].fillna(method='ffill').fillna(method='bfill')


In [126]:
l_foot_contact = selfpace_data['L Foot Contact']
r_foot_contact = selfpace_data['R Foot Contact']

In [127]:
# Detect heel strikes (contact onset) and toe-offs (contact offset)
heel_strike_l = l_foot_contact.diff() > 0  # Heel strike for left foot
toe_off_l = l_foot_contact.diff() < 0      # Toe-off for left foot

In [128]:
heel_strike_r = r_foot_contact.diff() > 0  # Heel strike for right foot
toe_off_r = r_foot_contact.diff() < 0      # Toe-off for right foot

In [129]:
# Get indices of heel strikes for segmentation
heel_strike_l_indices = heel_strike_l[heel_strike_l].index
heel_strike_r_indices = heel_strike_r[heel_strike_r].index

In [130]:
# 2.2 Segment into gait cycles
gait_cycles = []
for i in range(len(heel_strike_l_indices) - 1):
    start = heel_strike_l_indices[i]
    end = heel_strike_l_indices[i + 1]
    gait_cycles.append(selfpace_data.iloc[start:end])

In [131]:
# Step 2.3: Calculate spatiotemporal metrics
stride_times = []
stride_lengths = []
step_lengths = []

In [132]:
for cycle in gait_cycles:
    if len(cycle) > 1:
        # Stride time: Time difference between consecutive heel strikes
        stride_time = cycle['Time'].iloc[-1] - cycle['Time'].iloc[0]
        stride_times.append(stride_time)
        
        # Stride length: Euclidean distance between initial and final positions
        start_x, start_y = walkway_x_patterns.iloc[cycle.index[0]].mean(), walkway_y_patterns.iloc[cycle.index[0]].mean()
        end_x, end_y = walkway_x_patterns.iloc[cycle.index[-1]].mean(), walkway_y_patterns.iloc[cycle.index[-1]].mean()
        stride_length = np.sqrt((end_x - start_x) ** 2 + (end_y - start_y) ** 2)
        stride_lengths.append(stride_length)
        
        # Step length: Distance between left and right footfalls (mid-gait cycle)
        if len(cycle) > 2:
            mid_x, mid_y = walkway_x_patterns.iloc[cycle.index[len(cycle) // 2]].mean(), walkway_y_patterns.iloc[cycle.index[len(cycle) // 2]].mean()
            step_length = np.sqrt((mid_x - start_x) ** 2 + (mid_y - start_y) ** 2)
            step_lengths.append(step_length)


In [133]:
def compute_statistics(metrics):
    return {
        "Mean": float(np.mean(metrics)),
        "Std Dev": float(np.std(metrics)),
        "Coefficient of Variation": float(np.std(metrics) / np.mean(metrics)) if np.mean(metrics) != 0 else None
    }

In [134]:
stride_time_stats = compute_statistics(stride_times)
stride_length_stats = compute_statistics(stride_lengths)
step_length_stats = compute_statistics(step_lengths)

In [135]:
def symmetry_index(left, right):
    return float(abs(left - right) / ((left + right) / 2) * 100) if (left + right) != 0 else None


In [136]:
left_stride_lengths = stride_lengths[::2]  # Assuming alternating footfalls
right_stride_lengths = stride_lengths[1::2]
symmetry_indices = [
    symmetry_index(left, right) for left, right in zip(left_stride_lengths, right_stride_lengths)
]

In [137]:
# Step 4: Compare with Reference Data
# reference_stride_time = 1.0  # Example reference value (seconds)
# reference_stride_length = 1.5  # Example reference value (meters)

# stride_time_comparison = {
#     "Extracted": stride_time_stats,
#     "Reference": reference_stride_time
# }

# stride_length_comparison = {
#     "Extracted": stride_length_stats,
#     "Reference": reference_stride_length
# }

In [138]:
stride_results = {
    "Stride Time Stats": stride_time_stats,
    "Stride Length Stats": stride_length_stats,
    "Step Length Stats": step_length_stats,
    "Symmetry Indices": symmetry_indices,
    # "Comparisons": {
    #     "Stride Time": stride_time_comparison,
    #     "Stride Length": stride_length_comparison
    # }
}

# stride_results

{'Stride Time Stats': {'Mean': 2.0081818181818187,
  'Std Dev': 1.535920323346697,
  'Coefficient of Variation': 0.764831306329274},
 'Stride Length Stats': {'Mean': 116.35220312840455,
  'Std Dev': 15.182334252694593,
  'Coefficient of Variation': 0.13048600580377145},
 'Step Length Stats': {'Mean': 80.74687053884648,
  'Std Dev': 37.52443961338148,
  'Coefficient of Variation': 0.46471695265674556},
 'Symmetry Indices': [2.9140629767037534,
  15.434645641714994,
  28.35193610953835,
  0.3666132730682915,
  29.685240457324852]}

In [139]:
def fft_filter(data, fs, cutoff):
    """
    Filters the data using FFT by zeroing out frequencies above the cutoff.

    Parameters:
    - data: Array-like, the signal to filter.
    - fs: Sampling frequency in Hz.
    - cutoff: Cutoff frequency in Hz.

    Returns:
    - Filtered signal in the time domain.
    """
    # Perform FFT
    n = len(data)
    freq = fftfreq(n, d=1/fs)  # Frequency range
    fft_vals = fft(data)

    # Zero out frequencies beyond the cutoff
    fft_vals[np.abs(freq) > cutoff] = 0

    # Perform inverse FFT to reconstruct the signal
    filtered_signal = np.real(ifft(fft_vals))
    return filtered_signal

In [5]:
# Drop rows with NaN in relevant columns for CoP and FreeAcc
cop_free_acc_data = selfpace_data[['LCoP_X','LCoP_Y','RCoP_X', 'RCoP_Y', 'Rinsole:Acc_X', 'Rinsole:Acc_Y', 'Rinsole:Acc_Z']].dropna()

# Step 2.4: Extract Balance and Stability Metrics

# # Extract relevant columns for CoP (Center of Pressure) and FreeAcc (Free Acceleration)
# cop_x = cop_free_acc_data['RCoP_X']
# cop_y = cop_free_acc_data['RCoP_Y']
cop_x = pd.concat([cop_free_acc_data['LCoP_X'], cop_free_acc_data['RCoP_X']], axis=0)
cop_y = pd.concat([cop_free_acc_data['LCoP_Y'], cop_free_acc_data['RCoP_Y']], axis=0)
free_acc_x = cop_free_acc_data['Rinsole:Acc_X']
free_acc_y = cop_free_acc_data['Rinsole:Acc_Y']
free_acc_z = cop_free_acc_data['Rinsole:Acc_Z']

cop_x = cop_x.to_numpy()
cop_y = cop_y.to_numpy()
free_acc_x = free_acc_x.to_numpy()
free_acc_y = free_acc_y.to_numpy()
free_acc_z = free_acc_z.to_numpy()

## FILTERED

# Parameters for FFT filtering
sampling_frequency = 100
cutoff_frequency = 5  # Cutoff frequency in Hz

cop_x = fft_filter(cop_x, sampling_frequency, cutoff_frequency)
cop_y = fft_filter(cop_y, sampling_frequency, cutoff_frequency)
free_acc_x = fft_filter(free_acc_x, sampling_frequency, cutoff_frequency)
free_acc_y = fft_filter(free_acc_y, sampling_frequency, cutoff_frequency)
free_acc_z = fft_filter(free_acc_z, sampling_frequency, cutoff_frequency)


SyntaxError: invalid syntax (<ipython-input-5-3cb5574108c3>, line 9)

In [141]:
# Compute sway indices (e.g., path length, area of sway)
# Path length of CoP (sum of stepwise distances)
cop_path_length = float(np.sum(np.sqrt(np.diff(cop_x) ** 2 + np.diff(cop_y) ** 2)))

# Area of sway (approximate as bounding box area)
cop_area_sway = float((cop_x.max() - cop_x.min()) * (cop_y.max() - cop_y.min()))

# Stability index based on FreeAcc
stability_index = float(np.sqrt(free_acc_x.var() + free_acc_y.var() + free_acc_z.var()))



In [142]:
# Step 3: Statistical Analysis for Sway and Stability

# Compute variability metrics for sway and stability indices
sway_stats = {
    "Path Length": cop_path_length,
    "Sway Area": cop_area_sway,
    "Stability Index": stability_index
}

In [143]:
variability_metrics = {
    "Sway Path Length": compute_statistics(np.diff(cop_x) ** 2 + np.diff(cop_y) ** 2),
    "FreeAcc Stability": compute_statistics([free_acc_x.var(), free_acc_y.var(), free_acc_z.var()])
}

In [144]:
# Step 4: Compare with Reference Data
# Define reference values for sway and stability metrics (example values)
reference_sway_path_length = 0.5  # Example reference for path length
reference_stability_index = 0.1  # Example reference for stability index

sway_comparison = {
    "Extracted": sway_stats,
    "Reference": {
        "Path Length": reference_sway_path_length,
        "Stability Index": reference_stability_index
    }
}

In [149]:
# Final Results Compilation


# final_results = {
#     "spatiotemporal_metrics": {
#         "Stride Time": stride_time_stats,
#         "Stride Length": stride_length_stats,
#         "Step Length": step_length_stats
#     },
#     "symmetry_indices": symmetry_indices,
#     "balance_and_stability_metrices": sway_stats,
#     "variability_metrics": variability_metrics,
#     # "Comparisons": sway_comparison
# }

# final results separated
final_results = {
    "stride_time_mean":stride_time_stats['Mean'],
    "stride_time_std_dev":stride_time_stats['Std Dev'],
    "stride_time_CoV":stride_time_stats['Coefficient of Variation'],
    "stride_length_mean":stride_length_stats['Mean'],
    "stride_length_std_dev":stride_length_stats['Std Dev'],
    "stride_length_CoV":stride_length_stats['Coefficient of Variation'],
    "step_length_mean":step_length_stats['Mean'],
    "step_length_std_dev":step_length_stats['Std Dev'],
    "step_length_CoV":step_length_stats['Coefficient of Variation'],
    "symmetry_indices": symmetry_indices,
    "path_length": sway_stats['Path Length'],
    "sway_area": sway_stats['Sway Area'],
    "stability_index": sway_stats['Stability Index'],
    "sway_path_length_mean":variability_metrics['Sway Path Length']['Mean'],
    "sway_path_length_dev":variability_metrics['Sway Path Length']['Std Dev'],
    "sway_path_length_CoV":variability_metrics['Sway Path Length']['Coefficient of Variation'],
    "free_acc_stability_mean":variability_metrics['FreeAcc Stability']['Mean'],
    "free_acc_stability_dev":variability_metrics['FreeAcc Stability']['Std Dev'],
    "free_acc_stability_CoV":variability_metrics['FreeAcc Stability']['Coefficient of Variation'],

    # "variability_metrics": variability_metrics,
    # "Comparisons": sway_comparison
}



# final_results

{'stride_time_mean': 2.0081818181818187,
 'stride_time_std_dev': 1.535920323346697,
 'stride_time_CoV': 0.764831306329274,
 'stride_length_mean': 116.35220312840455,
 'stride_length_std_dev': 15.182334252694593,
 'stride_length_CoV': 0.13048600580377145,
 'step_length_mean': 80.74687053884648,
 'step_length_std_dev': 37.52443961338148,
 'step_length_CoV': 0.46471695265674556,
 'symmetry_indices': [2.9140629767037534,
  15.434645641714994,
  28.35193610953835,
  0.3666132730682915,
  29.685240457324852],
 'path_length': 72.68338576329901,
 'sway_area': 0.33523216451012816,
 'stability_index': 0.7988712985515577,
 'sway_path_length_mean': 0.00015311900469644184,
 'sway_path_length_dev': 0.0002885424641316676,
 'sway_path_length_CoV': 1.8844327306313318,
 'free_acc_stability_mean': 0.2127317838831507,
 'free_acc_stability_dev': 0.15453589575424284,
 'free_acc_stability_CoV': 0.7264353870088651}